##Datasets Auxiliares

In [ ]:
pip install googletrans==3.1.0a0

## 0. Librerías

In [ ]:
# Data load and manipulation
from google.colab import files
import io
 
# DataFrame librery
import pandas as pd
 
# Visualization 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Basic Operations
import numpy as np
import itertools
import operator
import re
 
# Prepocessing
from sklearn import preprocessing 
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer

#Translate
from googletrans import Translator

## 1. Carga de datos

Se cargarán dos dataset, **steam_games**, que hace referencia al dataset principal, y **steam_requirements_data**, el dataset de apoyo del que intentaremos extraer los máximos datos posibles.

In [ ]:
steam_games = pd.read_csv('steam.csv', sep=',') ## Dataset principal
game_requirements = pd.read_csv('steam_requirements_data.csv', sep=',') ## Dataset de apoyo


## 2. Dataset principal

In [ ]:
print("Número de registros: " + str(len(steam_games)))
steam_games.head()

Número de registros: 27075


,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


Se pretende adquirir información adicional de los requisitos hardware de cada juego.

## 3. Dataset de apoyo

Este dataset cuenta con los requisitos minimos y recomendados para jugar cada juego segun el sistema operativo y caracteristicas generales idependientes de estos.

Debido a que vamos a modificar los DataFrame, vamos a crear una copia para poder tener a mano los datos originales aún modificando estos DataFrame.

In [ ]:
print("Número de registros en el dataset original: " + str(len(game_requirements)))
game_requirements.head()

Número de registros en el dataset original: 27319


,steam_appid,pc_requirements,mac_requirements,linux_requirements,minimum,recommended
0,10,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
1,20,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
2,30,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
3,40,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
4,50,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN


In [ ]:
original_game_requirements = game_requirements.copy()

### Localización de los valores vacíos

In [ ]:
pd.isnull(game_requirements).any()

steam_appid           False
pc_requirements       False
mac_requirements      False
linux_requirements    False
minimum                True
recommended            True
dtype: bool

In [ ]:
print("Número de valores vacíos por variable en los datasets auxiliares: ")
game_requirements.isnull().sum()

Número de valores vacíos por variable en los datasets auxiliares: 


steam_appid               0
pc_requirements           0
mac_requirements          0
linux_requirements        0
minimum                   5
recommended           13185
dtype: int64

### Limpieza del dataset

Depuramos el dataset, eliminando columnas innecesarias y registros que no tienen un valor real. 

In [ ]:
## Columnas que no aportan nada completas.
game_requirements.drop("recommended", axis = 1, inplace = True) ## Columna vacía por defecto.

game_requirements

,steam_appid,pc_requirements,mac_requirements,linux_requirements,minimum
0,10,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ..."
1,20,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ..."
2,30,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ..."
3,40,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ..."
4,50,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ..."
...,...,...,...,...,...
27314,1065230,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],OS: Windows 7 Processor: Inter Core i7 Memory:...
27315,1065570,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"OS: Windows XP, Vista, 7, 8, 10 Processor: Int..."
27316,1065650,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],"OS: Windows 7, Windows 8, Windows 10 (32/64bit..."
27317,1066700,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],OS: Windows XP or later Processor: 1.5 GHz Mem...


### Preprocesado del resto de las variables del dataset de soporte


Al dataset principal se le han agregado una serie datasets auxiliares, que dan soporte al principal.

Algunas de las variables se encuentran en distintintos idiomas, encontrandonos una gran variedad donde predomina el ingles, en adicción a esta situación nos encontramos con una variedad de formatos en los que se definen las carcateriscas. Para poder preprocesar la información se procede a traducir todos los idiomas detectados al ingles, eliminar etiquetas que no proporcionan informacion de utilidad dejando un formato que es posible analizar para obtener la información.

Definimos el traductor y las columnas de las que obtendremos la información.

In [ ]:
translator = Translator()
column = ['pc_requirements', 'minimum','linux_requirements']

Prestamos especial atención que la dificultad que tiene origen en la variedad de idiomas y lenguajes en la que se recoge la informacion impide captar toda la información en el dataset, por lo que nos centramos en el **procesador** *velocidad en Hz* y el tamaño de **RAM** *en MB*.

Creamos las columnas que contendran el resultado de analizar la información

In [ ]:
game_requirements["RAM"] = np.nan
game_requirements["RAM_MBytes"] = np.nan
game_requirements["Processor"] = np.nan
game_requirements["Processor MHZ"] = np.nan

Metodos requeridos:
> **aplanar(s)**: aplanamos la lista 's'y eliminamos la información repetida y las expresiones regulares que no aportan información util. La lista es la generada por la libreria de expresiones regulares.

> **getRam(i, column)**: analizamos y recogemos la información de la **RAM** perteneciente a una columna.

> **getProcessor(i, column)**: analizamos y recogemos la información del **Procesador** perteneciente a una columna.






In [ ]:
def aplanar(s, element_unseful):
  element_unseful = ['mb','gb','ram','RAM', '']
  s = [y for x in s for y in x]
  for e in element_unseful:
    s = list(filter(lambda a: a != e, s))
  return list(dict.fromkeys(s))

In [ ]:
def cleanHTML(i, column):
  #print(game_requirements.pc_requirements[i])
  s = re.sub(r'<[/a-zA-Z0-9_"=\' ' '\t]*>', " ", game_requirements[column][i])
  s = re.sub(r'\\t', " ", re.sub(r'\\r', "", re.sub(r'\\n', "", s)))
  #s = translator.translate(s, dest='en').text
  #print(s)
  game_requirements.loc[i, column] = s


In [ ]:
def getRam(i, column):
  s = re.findall(r'((\d+ ?((mb)|(gb)) (ram))|(\d+ ?((mb)|(gb))))', game_requirements[column][i].lower())
  element_unseful = ['mb','gb','ram','RAM', '']
  s = aplanar(s, element_unseful)
  if len(s) != 0:
    game_requirements.loc[i, 'RAM'] = s[0]
    try:
      if 'mb' in s[0]:
        game_requirements.loc[i, 'RAM_MBytes'] = (int)(re.findall(r'\d+', s[0])[0])
      else:
        game_requirements.loc[i, 'RAM_MBytes'] = (int)(re.findall(r'\d+', s[0])[0])*1000
    except:
      print('Error en el analisis de la memoria ram en el registro ', i)


In [ ]:
def getProcessor(i, column):
  s = game_requirements[column][i].split('Memory')[0]
  s = re.findall(r'(\d+((\.|\,)\d+)? ?(g|m)hz)|(processor: [\w0-9a-zA-Z-\. ]+)', s.lower())
  element_unseful = ['m','mhz','hz','g','ghz', '']
  s = aplanar(s, element_unseful)
  if len(s) != 0:
    s = re.sub(r'Memory', " ", s[0])
    game_requirements.loc[i, 'Processor'] = s
    try:
      if 'hz' in s:
        s = re.sub(r'\,', '\.', s)
        if 'ghz' in s:
          game_requirements.loc[i, 'Processor MHZ'] = ((float)(aplanar(re.findall(r'(\d+\.\d+)|(\d+)', s), element_unseful)[0]))*1000
        else:
          game_requirements.loc[i, 'Processor MHZ'] = (float)(aplanar(re.findall(r'(\d+\.\d+)|(\d+)', s), element_unseful)[0])
    except:
      print('Error en el analisis del procesador en el registro ', i)


In [ ]:
for i in game_requirements.index:
  try:
      cleanHTML(i, column[1])
      getRam(i, column[1])
      getProcessor(i, column[1])
  except:
    print('Error en el analisis en el registro ', i)

game_requirements.head()


Error en el analisis en el registro  6109
Error en el analisis en el registro  8471
Error en el analisis del procesador en el registro  9616
Error en el analisis en el registro  11043
Error en el analisis en el registro  13063
Error en el analisis en el registro  15587


,steam_appid,pc_requirements,mac_requirements,linux_requirements,minimum,RAM,RAM_MBytes,Processor,Processor MHZ
0,10,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",96mb ram,96.0,500 mhz,500.0
1,20,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",96mb ram,96.0,500 mhz,500.0
2,30,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",96mb ram,96.0,500 mhz,500.0
3,40,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",96mb ram,96.0,500 mhz,500.0
4,50,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",96mb ram,96.0,500 mhz,500.0


Respasamos los resultados del analisis y adquisición de la información aplicada sobre la columna de caracteristicas mínimas.

In [ ]:
print("Número de valores vacíos por variable en los datasets auxiliares: ")
game_requirements.isnull().sum()

Número de valores vacíos por variable en los datasets auxiliares: 


steam_appid               0
pc_requirements           0
mac_requirements          0
linux_requirements        0
minimum                   5
RAM                     412
RAM_MBytes              412
Processor              2006
Processor MHZ         14363
dtype: int64

Nos centramos en las posiciones que debido a la variedad de los lenguajes y a la incompletitud de la columna no se han podido recoger la información.

In [ ]:
is_NaN = game_requirements.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = game_requirements[row_has_NaN]
rows_with_NaN.head()

,steam_appid,pc_requirements,mac_requirements,linux_requirements,minimum,RAM,RAM_MBytes,Processor,Processor MHZ
25,730,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,OS: Windows® 7/Vista/XP Processor: Intel® Core...,2 gb ram,2000.0,processor: intel,NaN
31,1520,{'minimum': '<strong>Recommended:</strong> Win...,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...","{'minimum': '<ul class=""bb_ul""><li><strong>Pro...","Recommended: Windows XP, P3-600-Geforce 2, 128...",128 mb ram,128.0,NaN,NaN
34,1610,{'minimum': '<strong>Minimum:</strong> Pentium...,[],[],Pentium Processor; Windows XP; 128MB RAM; 800x...,128mb ram,128.0,NaN,NaN
38,1690,{'minimum': '<strong>Minimum System Requiremen...,[],[],Minimum System Requirements: Pentium 500MHz or...,NaN,NaN,500mhz,500.0
45,2210,"{'minimum': '<ul class=""bb_ul""><li><strong>Ope...",[],[],Operating System: Windows® XP Processor: Inte...,512mb ram,512.0,processor: intel,NaN


In [ ]:
print("Número de registros con valores nulos en el dataset: " + str(len(rows_with_NaN)))

Número de registros con valores nulos en el dataset: 14402


Se realiza una segunda pasada con el obtetivo de recuperar la mayor información posible de otras columnas, haciendo especial atencion en la de **"pc_requeriments"**.

In [ ]:
for i in rows_with_NaN.index:
  try:
    cleanHTML(i, column[0])
    getRam(i, column[0])
    getProcessor(i, column[0])
  except:
    print(i,'except')

Error en el analisis del procesador en el registro  9616


In [ ]:
pd.isnull(game_requirements).any()

steam_appid           False
pc_requirements       False
mac_requirements      False
linux_requirements    False
minimum                True
RAM                    True
RAM_MBytes             True
Processor              True
Processor MHZ          True
dtype: bool

In [ ]:
print("Número de valores vacíos por variable en los datasets auxiliares: ")
game_requirements.isnull().sum()

Número de valores vacíos por variable en los datasets auxiliares: 


steam_appid               0
pc_requirements           0
mac_requirements          0
linux_requirements        0
minimum                   5
RAM                     394
RAM_MBytes              394
Processor              1920
Processor MHZ         14349
dtype: int64

###Tratamiento de valores nulos

Como a pesar de los metodos de obtencion de información aun residen valores nulos se rellenan las columnas de **'Processor MHZ'** y **'RAM_MBytes'** que contienen valores numericos con la media de estos.

In [ ]:
game_requirements['RAM_MBytes'].fillna(game_requirements['RAM_MBytes'].mean(), inplace = True)

In [ ]:
pd.isnull(game_requirements).any()

steam_appid           False
pc_requirements       False
mac_requirements      False
linux_requirements    False
minimum                True
RAM                    True
RAM_MBytes            False
Processor              True
Processor MHZ         False
dtype: bool

### Cálculo de la media

In [ ]:
 ram_mean = game_requirements.loc[:,'RAM_MBytes'].mean()
 ram_mean

2712.8930733518864

In [ ]:
proc_mean = game_requirements.loc[:,'Processor MHZ'].mean()
proc_mean

46260.17317346671

### Cálculo de la mediana

In [ ]:
ram_median = game_requirements.loc[:,'RAM_MBytes'].median()
ram_median

2000.0

In [ ]:
proc_median = game_requirements.loc[:,'Processor MHZ'].median()
proc_median

46260.17317347726

### Cálculo de la desviación típica

Además, también vamos a calcular la desviación típica, que nos ofrecerá información sobre la dispersión media de la RAM requerida y del Procesador de cada juego.

In [ ]:
# Agrupamos por desviación típica
ram_std = game_requirements.loc[:,'RAM_MBytes'].std(ddof=0)
ram_std

17251.404860834657

In [ ]:
# Agrupamos por desviación típica
proc_std = game_requirements.loc[:,'Processor MHZ'].std(ddof=0)
proc_std

2456589.9358918187

###Exportando los datos 

Mediante la siguiente función, podremos guardar y exportar los datos calculados en formato CSV, de tal forma, que se puedan cargar en otros cuadernos para seguir con el desarrollo de este estudio.

In [ ]:
game_requirements.to_csv('auxiliar1.csv')